<a href="https://colab.research.google.com/github/syedaftab-dev/Machine-learning/blob/main/Adaboost1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Random Forest Classification implementaion


## later we did adaboost in this only

## Holiday Package Prediction

"Travel.Com" company wants to enable and establish a viable business model to expand the customer base. One of the ways to expand the customer base is to introduce a new offering of packages. Currently, there are 5 types of packages the company is offering - Basic, Standard, Deluxe, Super Deluxe, King. Looking at the data of the last year, we observed that 18% of the customers purchased the packages. However, the marketing cost was quite high because customers were contacted at random without looking at the available information. The company is now planning to launch a new product i.e. Wellness Tourism Package. Wellness Tourism is defined as Travel that allows the traveler to maintain, enhance or kick-start a healthy lifestyle, and support or increase one's sense of well-being.However, this time company wants to harness the available data of existing and potential customers to make the marketing expenditure more efficient.

We need to analyze the customers' data and information to provide recommendations to the Policy Maker and Marketing Team and also build a model to predict the potential customer who is going to purchase the newly introduced travel package.


In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

In [3]:
# Install dependencies as needed:
# pip install kagglehub[pandas-datasets]
import kagglehub
from kagglehub import KaggleDatasetAdapter

# Set the path to the file you'd like to load
file_path = "Travel.csv"

# Load the latest version
df = kagglehub.load_dataset(
  KaggleDatasetAdapter.PANDAS,
  "susant4learning/holiday-package-purchase-prediction",
  file_path,
  # Provide any additional arguments like
  # sql_query or pandas_kwargs. See the
  # documenation for more information:
  # https://github.com/Kaggle/kagglehub/blob/main/README.md#kaggledatasetadapterpandas
)

print("First 5 records:", df.head())

/tmp/ipython-input-3546803341.py:10: DeprecationWarning: load_dataset is deprecated and will be removed in a future version.
  df = kagglehub.load_dataset(


100%|██████████| 458k/458k [00:00<00:00, 532kB/s]

First 5 records:    CustomerID  ProdTaken   Age    TypeofContact  CityTier  DurationOfPitch  \
0      200000          1  41.0     Self Enquiry         3              6.0   
1      200001          0  49.0  Company Invited         1             14.0   
2      200002          1  37.0     Self Enquiry         1              8.0   
3      200003          0  33.0  Company Invited         1              9.0   
4      200004          0   NaN     Self Enquiry         1              8.0   

       Occupation  Gender  NumberOfPersonVisiting  NumberOfFollowups  \
0        Salaried  Female                       3                3.0   
1        Salaried    Male                       3                4.0   
2     Free Lancer    Male                       3                4.0   
3        Salaried  Female                       2                3.0   
4  Small Business    Male                       2                3.0   

  ProductPitched  PreferredPropertyStar MaritalStatus  NumberOfTrips  \
0        

In [4]:
df.head()

,CustomerID,ProdTaken,Age,TypeofContact,CityTier,DurationOfPitch,Occupation,Gender,NumberOfPersonVisiting,NumberOfFollowups,ProductPitched,PreferredPropertyStar,MaritalStatus,NumberOfTrips,Passport,PitchSatisfactionScore,OwnCar,NumberOfChildrenVisiting,Designation,MonthlyIncome
0,200000,1,41.0,Self Enquiry,3,6.0,Salaried,Female,3,3.0,Deluxe,3.0,Single,1.0,1,2,1,0.0,Manager,20993.0
1,200001,0,49.0,Company Invited,1,14.0,Salaried,Male,3,4.0,Deluxe,4.0,Divorced,2.0,0,3,1,2.0,Manager,20130.0
2,200002,1,37.0,Self Enquiry,1,8.0,Free Lancer,Male,3,4.0,Basic,3.0,Single,7.0,1,3,0,0.0,Executive,17090.0
3,200003,0,33.0,Company Invited,1,9.0,Salaried,Female,2,3.0,Basic,3.0,Divorced,2.0,1,5,1,1.0,Executive,17909.0
4,200004,0,NaN,Self Enquiry,1,8.0,Small Business,Male,2,3.0,Basic,4.0,Divorced,1.0,0,5,1,0.0,Executive,18468.0


Data Cleaning

handling missing values

In [5]:
df.isnull().sum()

,0
CustomerID,0
ProdTaken,0
Age,226
TypeofContact,25
CityTier,0
DurationOfPitch,251
Occupation,0
Gender,0
NumberOfPersonVisiting,0
NumberOfFollowups,45


In [6]:
# check all the categories
df['Gender'].value_counts()

,count
Gender,
Male,2916
Female,1817
Fe Male,155


In [7]:
df['MaritalStatus'].value_counts()
# make single and umarried as one

,count
MaritalStatus,
Married,2340
Divorced,950
Single,916
Unmarried,682


In [8]:
df['TypeofContact'].value_counts()

,count
TypeofContact,
Self Enquiry,3444
Company Invited,1419


In [9]:
df['MaritalStatus']=df['MaritalStatus'].replace('single','Unmarried')
df['Gender'] = df['Gender'].replace('Fe Male','Female')

In [10]:
df.head()

,CustomerID,ProdTaken,Age,TypeofContact,CityTier,DurationOfPitch,Occupation,Gender,NumberOfPersonVisiting,NumberOfFollowups,ProductPitched,PreferredPropertyStar,MaritalStatus,NumberOfTrips,Passport,PitchSatisfactionScore,OwnCar,NumberOfChildrenVisiting,Designation,MonthlyIncome
0,200000,1,41.0,Self Enquiry,3,6.0,Salaried,Female,3,3.0,Deluxe,3.0,Single,1.0,1,2,1,0.0,Manager,20993.0
1,200001,0,49.0,Company Invited,1,14.0,Salaried,Male,3,4.0,Deluxe,4.0,Divorced,2.0,0,3,1,2.0,Manager,20130.0
2,200002,1,37.0,Self Enquiry,1,8.0,Free Lancer,Male,3,4.0,Basic,3.0,Single,7.0,1,3,0,0.0,Executive,17090.0
3,200003,0,33.0,Company Invited,1,9.0,Salaried,Female,2,3.0,Basic,3.0,Divorced,2.0,1,5,1,1.0,Executive,17909.0
4,200004,0,NaN,Self Enquiry,1,8.0,Small Business,Male,2,3.0,Basic,4.0,Divorced,1.0,0,5,1,0.0,Executive,18468.0


In [11]:
df["Gender"].value_counts()

,count
Gender,
Male,2916
Female,1972


In [12]:
## checking missing values

features_with_na = [features for features in df.columns if df[features].isnull().sum()>=1]
for feature in features_with_na:
  print(feature,np.round(df[feature].isnull().mean()*100,5),'% missing values')


Age 4.62357 % missing values
TypeofContact 0.51146 % missing values
DurationOfPitch 5.13502 % missing values
NumberOfFollowups 0.92062 % missing values
PreferredPropertyStar 0.53191 % missing values
NumberOfTrips 2.86416 % missing values
NumberOfChildrenVisiting 1.35025 % missing values
MonthlyIncome 4.76678 % missing values


In [13]:
# stattics on numerical columns
df[features_with_na].select_dtypes(exclude='object').describe()
# select_dtypes(exclude='object') method to ensure only numerical columns within that list are considered

,Age,DurationOfPitch,NumberOfFollowups,PreferredPropertyStar,NumberOfTrips,NumberOfChildrenVisiting,MonthlyIncome
count,4662.000000,4637.000000,4843.000000,4862.000000,4748.000000,4822.000000,4655.000000
mean,37.622265,15.490835,3.708445,3.581037,3.236521,1.187267,23619.853491
std,9.316387,8.519643,1.002509,0.798009,1.849019,0.857861,5380.698361
min,18.000000,5.000000,1.000000,3.000000,1.000000,0.000000,1000.000000
25%,31.000000,9.000000,3.000000,3.000000,2.000000,1.000000,20346.000000
50%,36.000000,13.000000,4.000000,3.000000,3.000000,1.000000,22347.000000
75%,44.000000,20.000000,4.000000,4.000000,4.000000,2.000000,25571.000000
max,61.000000,127.000000,6.000000,5.000000,22.000000,3.000000,98678.000000


# Imputting Null Values
1. impute median values for age column and other numerical features

In [14]:
data = pd.Series([1, 2, 2, 3, 3, 2])
mode_value = data.mode()[0]
print(mode_value)
# if there are multiple modes (e.g., a tie in frequency), mode() returns all of them in sorted order, and [0] picks the first.
# in this case mode = [2,2] so we took the first one

2


In [15]:
# age
df['Age'].fillna(df.Age.median(),inplace=True)

#TypeOf cOntract mode for categorical features
df['TypeofContact'].fillna(df.TypeofContact.mode()[0],inplace=True)

# duration Pitch
df.DurationOfPitch.fillna(df.DurationOfPitch.median(),inplace=True)
# NumberOfFollowers
df.NumberOfFollowups.fillna(df.NumberOfFollowups.mode()[0],inplace=True)
#preferredPropertyStart
df.PreferredPropertyStar.fillna(df.PreferredPropertyStar.mode()[0],inplace=True)
#numberofTrips
df.NumberOfTrips.fillna(df.NumberOfTrips.median(),inplace=True)
#NumberofChildrenvisiting
df.NumberOfChildrenVisiting.fillna(df.NumberOfChildrenVisiting.mode()[0],inplace=True)
# monthlyIncome
df.MonthlyIncome.fillna(df.MonthlyIncome.median(),inplace=True)


/tmp/ipython-input-3489049084.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['Age'].fillna(df.Age.median(),inplace=True)
/tmp/ipython-input-3489049084.py:5: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try usin

In [16]:
df.drop('CustomerID',inplace=True,axis=1)
df.isnull().sum()

,0
ProdTaken,0
Age,0
TypeofContact,0
CityTier,0
DurationOfPitch,0
Occupation,0
Gender,0
NumberOfPersonVisiting,0
NumberOfFollowups,0
ProductPitched,0


# Feature Engineering

In [17]:
# create new column for feature
df['TotalVisiting']=df['NumberOfPersonVisiting']+df['NumberOfChildrenVisiting']
df.drop(columns=['NumberOfPersonVisiting','NumberOfChildrenVisiting'],inplace=True,axis=1)

In [18]:
# get all thehh numeric features
num_features = [feature for feature in df.columns if df[feature].dtype!='O']
print("Number of numerical features : ",len(num_features))
#

Number of numerical features :  12


In [19]:
 # categorical features
 cat_features = [feature for feature in df.columns if df[feature].dtype=='O']
 print("Number of categorical features : ",len(cat_features))

Number of categorical features :  6


In [20]:
# decrete features -> fix no of categories <=25
discrete_features = [feature for feature in num_features if len(df[feature].unique()<=25)]
print("Number of discrete features : ",len(discrete_features))

Number of discrete features :  12


In [21]:
  # continuous features
continuous_features = [feature for feature in num_features if feature not in discrete_features]
print("Number of continuous features : ",len(continuous_features))

Number of continuous features :  0


In [22]:
from sklearn.model_selection import train_test_split
X=df.drop(['ProdTaken'],axis=1)
y=df['ProdTaken']

In [23]:
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=42)

In [24]:
X_train.shape,X_test.shape


((3910, 17), (978, 17))

In [25]:
# create column transformer with 3 types of tranformers
cat_features=X.select_dtypes(include='object').columns
num_features=X.select_dtypes(exclude='object').columns

from sklearn.preprocessing import OneHotEncoder,StandardScaler
from sklearn.compose import ColumnTransformer # help to apply columtansformation for multiple type of like numerical,categorical at a time
numeric_transformer = StandardScaler() # for numerical features
oh_tranformer = OneHotEncoder(drop='first') # for categorical fetures

preprocessor = ColumnTransformer(
    [
        ("OneHotEncoder",oh_tranformer,cat_features),
        ("StandardScaler",numeric_transformer,num_features )
    ]
)


In [26]:
preprocessor

ColumnTransformer(transformers=[('OneHotEncoder', OneHotEncoder(drop='first'),
                                 Index(['TypeofContact', 'Occupation', 'Gender', 'ProductPitched',
       'MaritalStatus', 'Designation'],
      dtype='object')),
                                ('StandardScaler', StandardScaler(),
                                 Index(['Age', 'CityTier', 'DurationOfPitch', 'NumberOfFollowups',
       'PreferredPropertyStar', 'NumberOfTrips', 'Passport',
       'PitchSatisfactionScore', 'OwnCar', 'MonthlyIncome', 'TotalVisiting'],
      dtype='object'))])

In [27]:
X_train = preprocessor.fit_transform(X_train)
X_test = preprocessor.transform(X_test)

In [28]:
pd.DataFrame(X_train)

,0,1,2,3,4,5,6,7,8,9,...,17,18,19,20,21,22,23,24,25,26
0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,...,-0.721400,-1.020350,1.284279,-0.725271,-0.127737,-0.632399,0.679690,0.782966,-0.382245,-0.774151
1,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,...,-0.721400,0.690023,0.282777,-0.725271,1.511598,-0.632399,0.679690,0.782966,-0.459799,0.643615
2,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,-0.721400,-1.020350,0.282777,1.771041,0.418708,-0.632399,0.679690,0.782966,-0.245196,-0.065268
3,1.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,...,-0.721400,-1.020350,1.284279,-0.725271,-0.127737,-0.632399,1.408395,-1.277194,0.213475,-0.065268
4,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,-0.721400,2.400396,-1.720227,-0.725271,1.511598,-0.632399,-0.049015,-1.277194,-0.024889,2.061382
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3905,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,...,-0.721400,-0.653841,1.284279,-0.725271,-0.674182,-0.632399,-1.506426,0.782966,-0.536973,0.643615
3906,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,1.455047,-0.898180,-0.718725,1.771041,-1.220627,-0.632399,1.408395,0.782966,1.529609,-0.065268
3907,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.455047,1.545210,0.282777,-0.725271,2.058043,-0.632399,-0.777720,0.782966,-0.360576,0.643615
3908,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,...,1.455047,1.789549,1.284279,-0.725271,-0.127737,-0.632399,-1.506426,0.782966,-0.252799,0.643615


Random forest classifieer

In [36]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score,confusion_matrix,classification_report,precision_score,recall_score,f1_score,roc_auc_score

# using adabost
from sklearn.ensemble import AdaBoostClassifier

In [37]:
models = {
    "Logistic Regression": LogisticRegression(),
    "Decision Tree":DecisionTreeClassifier(),
    "Random Forest": RandomForestClassifier(),
    "AdaBoost":AdaBoostClassifier()
# contains lot of decision tree models
}

for i in range(len(list(models))):
  model=list(models.values())[i] # taking the values(since it was a dictionary) from model list [i]->index of model 1st 2nd ....

  model.fit(X_train,y_train)
  y_train_pred=model.predict(X_train)
  y_test_pred=model.predict(X_test)


  # trinaing set performance
  model_train_accuracy = accuracy_score(y_train,y_train_pred)
  model_train_f1 = f1_score(y_train,y_train_pred)
  model_train_recall = recall_score(y_train,y_train_pred)
  model_train_precision = precision_score(y_train,y_train_pred)
  model_train_roc = roc_auc_score(y_train,y_train_pred)

  # test set performance
  model_test_accuracy = accuracy_score(y_test,y_test_pred)
  model_test_f1 = f1_score(y_test,y_test_pred)
  model_test_recall = recall_score(y_test,y_test_pred)
  model_test_precision = precision_score(y_test,y_test_pred)
  model_test_roc = roc_auc_score(y_test,y_test_pred)

  print(list(models.keys())[i])
  print('model performance for training set')
  print('- accuracy: {:.4f}'.format(model_train_accuracy))
  print('- f1 score: {:.4f}'.format(model_train_f1))
  print('- recall: {:.4f}'.format(model_train_recall))
  print('- precision: {:.4f}'.format(model_train_precision))
  print('- roc auc score: {:.4f}'.format(model_train_roc))

  print('----------------------------------')

  print('Model performance for Test set')
  print('- accuracy: {:.4f}'.format(model_test_accuracy))
  print('- f1 score: {:.4f}'.format(model_test_f1))
  print('- recall: {:.4f}'.format(model_test_recall))
  print('- precision: {:.4f}'.format(model_test_precision))
  print('- roc auc score: {:.4f}'.format(model_test_roc))


Logistic Regression
model performance for training set
- accuracy: 0.8455
- f1 score: 0.4259
- recall: 0.3073
- precision: 0.6935
- roc auc score: 0.6381
----------------------------------
Model performance for Test set
- accuracy: 0.8374
- f1 score: 0.4260
- recall: 0.3089
- precision: 0.6860
- roc auc score: 0.6373
Decision Tree
model performance for training set
- accuracy: 1.0000
- f1 score: 1.0000
- recall: 1.0000
- precision: 1.0000
- roc auc score: 1.0000
----------------------------------
Model performance for Test set
- accuracy: 0.9100
- f1 score: 0.7672
- recall: 0.7592
- precision: 0.7754
- roc auc score: 0.8529
Random Forest
model performance for training set
- accuracy: 1.0000
- f1 score: 1.0000
- recall: 1.0000
- precision: 1.0000
- roc auc score: 1.0000
----------------------------------
Model performance for Test set
- accuracy: 0.9192
- f1 score: 0.7476
- recall: 0.6126
- precision: 0.9590
- roc auc score: 0.8031
AdaBoost
model performance for training set
- accuracy:

In [41]:
# hyperparameter training
rf_params={
    "max_depth":[5,8,15,None,10],
    "max_features":[5,7,'auto',8],
    "min_samples_split":[2,3,5,4],
    "n_estimators":[100,200,500,1000]
}

# adding adabost hyprtuning
adabost_pramams = {
    "n_estimators" :[50,60,70,80,90],# no of decision tree stumps
    "algorithm" :['SAMME','SAMME.R']
}

In [42]:
randomcv_models=[
    ("RF",RandomForestClassifier(),rf_params),
    ("AB",AdaBoostClassifier(),adabost_pramams)
]

In [43]:
from sklearn.model_selection import RandomizedSearchCV

In [44]:
model_param = {}
for name,model,params in randomcv_models:
  random = RandomizedSearchCV(estimator=model,param_distributions=params,n_iter=70,cv=3,verbose=2,n_jobs=-1)
  random.fit(X_train,y_train)
  model_param[name] = random.best_params_

for model_name in model_param:
  print(f"------------------Best Prams for {model_name}------------------")
  print(model_param[model_name])

Fitting 3 folds for each of 70 candidates, totalling 210 fits


/usr/local/lib/python3.12/dist-packages/sklearn/model_selection/_validation.py:528: FitFailedWarning: 
39 fits failed out of a total of 210.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
21 fits failed with the following error:
Traceback (most recent call last):
  File "/usr/local/lib/python3.12/dist-packages/sklearn/model_selection/_validation.py", line 866, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/usr/local/lib/python3.12/dist-packages/sklearn/base.py", line 1382, in wrapper
    estimator._validate_params()
  File "/usr/local/lib/python3.12/dist-packages/sklearn/base.py", line 436, in _validate_params
    validate_parameter_constraints(
  File "/usr/local/lib/python3.12/dist-packages/sklearn/utils/

Fitting 3 folds for each of 10 candidates, totalling 30 fits


/usr/local/lib/python3.12/dist-packages/sklearn/model_selection/_validation.py:528: FitFailedWarning: 
15 fits failed out of a total of 30.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
15 fits failed with the following error:
Traceback (most recent call last):
  File "/usr/local/lib/python3.12/dist-packages/sklearn/model_selection/_validation.py", line 866, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/usr/local/lib/python3.12/dist-packages/sklearn/base.py", line 1382, in wrapper
    estimator._validate_params()
  File "/usr/local/lib/python3.12/dist-packages/sklearn/base.py", line 436, in _validate_params
    validate_parameter_constraints(
  File "/usr/local/lib/python3.12/dist-packages/sklearn/utils/_

------------------Best Prams for RF------------------
{'n_estimators': 500, 'min_samples_split': 2, 'max_features': 8, 'max_depth': None}
------------------Best Prams for AB------------------
{'n_estimators': 60, 'algorithm': 'SAMME'}
